### Mass segregation

Analysis of the mass segregation in the stellar clusters using different indices <br>
In particular we use the MST-based metric (e.g. Yu et al. 2011)

In [186]:
using PyCall
using Distributions
using Statistics
using DataFrames
using Distances, LightGraphs, SimpleWeightedGraphs
using Random , Printf

rootdir = "/home/stephane/Science/GAIA"

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering

import PyPlot , CSV

@pyimport astroquery.vizier as vizier
@pyimport astropy.units as u
@pyimport astropy.coordinates as coord


## directory
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/test"
ocdir= "$wdir/oc"

cd(wdir)

In [187]:
## Test OC
##

oc= CSV.read("$ocdir/NGC 6124-2.3deg-oc.csv" , delim= ";")

,ra,dec,l,b,distance,pmra,pmdec,X,Y,Z,vl,vb,vrad,gbar,rp,bp
,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,248.314,-39.1181,342.912,5.9616,172.377,-11.8593,-22.0303,172.245,6.74447,0.122253,-19.8159,-5.02337,NaN,21.765,14.5745,17.9966
2,247.712,-42.5732,340.058,3.94601,175.615,-14.0032,-21.5762,175.501,-1.8729,-6.05217,-21.0746,-3.78274,NaN,23.5374,16.2922,20.0264
3,247.984,-42.4081,340.317,3.91201,182.366,-13.1387,-20.4854,182.251,-1.12312,-6.39299,-20.6992,-3.75543,NaN,22.9287,15.8706,19.4002
4,247.993,-42.1859,340.484,4.05878,176.536,-12.7136,-20.2254,176.441,-0.572013,-5.73663,-19.6369,-3.74155,NaN,20.1256,13.2496,15.6377
5,247.991,-42.1857,340.483,4.06,179.973,-12.207,-21.2668,179.877,-0.585808,-5.84449,-20.3749,-4.73645,NaN,19.6776,12.8861,15.1388
6,247.6,-38.7443,342.809,6.62348,178.495,-14.3495,-21.87,178.357,6.66125,2.1885,-21.8137,-3.73251,NaN,23.0481,15.9003,19.3954
7,246.336,-39.0719,341.888,7.11242,181.983,-12.0267,-20.1924,181.903,3.86828,3.78401,-19.7453,-4.59687,NaN,23.6782,16.5324,20.3177
8,247.668,-41.0375,341.16,5.01999,173.786,-14.0097,-21.1868,173.758,1.48651,-2.73268,-20.6282,-3.4996,NaN,21.8272,14.7329,17.8784
9,247.994,-41.2395,341.18,4.70271,172.12,-12.4389,-20.9532,172.075,1.5313,-3.65946,-19.4317,-4.19825,NaN,21.1651,13.8491,16.7635


In [188]:
## Create a graph with the selected stars
## ind: subset to select the x,y
## dtyp= "2d" or "3d"

function _mst_graph(ind,x,y,z=[] , dtyp="2d")
    nxy= length(ind)
    if dtyp == "2d"
        A= Array{Float64}(undef,2,nxy)
    elseif dtyp == "3d"
       A= Array{Float64}(undef,3,nxy) 
    end
    
    for i in 1:nxy
        if dtyp == "2d"
            A[:,i]= [x[ind[i]] y[ind[i]]]
        elseif dtyp == "3d"
            A[:,i]= [x[ind[i]] y[ind[i]] z[ind[i]]]
        end
    end
    
    d= Euclidean()
    p= pairwise(d, A, A, dims=2)
    println(p)
    
    src= Array{Int}(undef,0)
    dst=  Array{Int}(undef,0)
    wgt= Array{Float64}(undef,0)

    for i in 1:nxy
        for j in 1:nxy
            if i != j
                push!(src,i)
                push!(dst,j)
                push!(wgt,p[i,j]) 
            end
        end
    end
    
    G= SimpleWeightedGraph(src, dst, wgt)
    
    kr= kruskal_mst(G, G.weights)
    
    for w in kr
        @printf("%d - %d : %3.3f\n",w.src,w.dst, w.weight)
    end
    
    return(kr)
end


_mst_graph (generic function with 3 methods)

In [189]:
## Computation of the λMST using either arithmetic or geometric average (Olczak et al. 2011)
##
## dtype= "ari" or "geo"
function _lambda_mst(edges, dtyp="ari")
    nedg= length(edges)
    println(nedg)
    ltot= 0
    
    for w in edges
        if dtyp== "ari"
            ltot+= w.weight
        elseif dtyp== "geo"
            ltot+= log(w.weight)
        end
    end
    
    if dtyp== "ari"
        λmst= ltot/nedg
    elseif dtyp== "geo"
        λmst= exp(ltot/nedg)
    end
    return(λmst)
end

## estimate the sample size for the stats given n and N for the massive and total sample
## p= 99%
## Olczak et al. 2011)
function _sample_size(n, N, p=0.99)
    k= log(1-p)/log(1-n/N)
    return(convert(Int,ceil(k)))
end

## Compute the mass segregation metric comparing two population set (2nd is random)
function _kappa_ms(indMass, x, y , z, dtyp="2d", daver= "geo")
    npop= length(indMass)
    ntot= length(x)
    
    ## Massive stars
    krMass= _mst_graph(indMass, x,y,z , dtyp)
    λMass= _lambda_mst(krMass,daver)
    ## Random set
    perm= randperm(ntot)
    indRef= perm[1:npop]
    krRef= _mst_graph(indRef, x,y,z, dtyp)
    λRef= _lambda_mst(krRef,daver)
    
    ## ratio (log) of the reference over massive ones
    κms= log(λRef/λMass)
    return(κms)
end

_kappa_ms (generic function with 3 methods)

In [190]:
## Main

# kr= _mst_graph([1,5,10,16,20,27,30], oc[:X], oc[:Y] , oc[:Z] , "2d")
#  _lambda_mst(kr,"geo")
# _sample_size(20, 100)

_kappa_ms( [1,5,10,16,20,27,30,35,45,56,67,78,89], oc[:X], oc[:Y] , oc[:Z], "3d", "ari")

[0.0 12.1479 5.68323 9.68944 7.09485 10.6536 10.0874 7.19032 9.33167 16.7929 16.7021 7.78216 6.80383; 12.1479 0.0 7.36595 5.97413 5.53305 8.28865 8.31821 13.0598 6.28658 12.0823 9.08029 5.537 6.36123; 5.68323 7.36595 0.0 4.31177 3.06889 6.10422 6.37278 7.56472 3.92683 11.8237 11.3522 5.39496 4.84071; 9.68944 5.97413 4.31177 0.0 4.14782 2.75675 4.00551 11.2018 0.577326 8.07724 7.06526 6.9181 6.77685; 7.09485 5.53305 3.06889 4.14782 0.0 5.96458 5.25351 10.2115 4.16456 12.1939 10.621 3.02787 2.65388; 10.6536 8.28865 6.10422 2.75675 5.96458 0.0 2.62395 12.9289 2.90126 6.87807 6.549 8.83235 8.50428; 10.0874 8.31821 6.37278 4.00551 5.25351 2.62395 0.0 13.7351 4.26475 9.12066 7.98749 7.75964 7.34187; 7.19032 13.0598 7.56472 11.2018 10.2115 12.9289 13.7351 0.0 10.6951 16.9278 17.5464 11.0902 10.656; 9.33167 6.28658 3.92683 0.577326 4.16456 2.90126 4.26475 10.6951 0.0 8.18097 7.45441 7.00263 6.80727; 16.7929 12.0823 11.8237 8.07724 12.1939 6.87807 9.12066 16.9278 8.18097 0.0 4.56161 14.8694 14.

0.3209886912422127